# Only modify the next cell

In [1]:
# Full path to the .csv file containing the narratives
fn = "/Users/schabdachj/Data/clinical_healthy/tables/rawdata/healthy_mr_cohort_jmy_annotations.csv"

# Phrase indicating non-healthy patients
phrase = "glioma"

# Don't modify this bit

In [2]:
from IPython.display import clear_output
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [3]:
def markPhraseUnusable(df, fn, phrase):
    
    count = 0
    
    for idx, row in df.iterrows():
        if phrase in row['narrative_text'].lower() and not type(row['confirm_healthy']) == bool:
            
            validInput = False
            
            while not validInput:
            
                print(row['narrative_text'].lower().replace(phrase, '\033[91m'+phrase.upper()+'\033[0m'))
                print()

                confirm = input("Can you confirm the patient is not healthy (y/n/save/exit)?")

                if confirm.lower() == "y": #or confirm.lower() == "":
                    df['confirm_healthy'].iloc[idx] = False
                    df['reason'].iloc[idx] = phrase
                    clear_output()
                    validInput = True

                elif confirm.lower() == "save":
                    clear_output()
                    print("Saving current dataframe state")
                    df.to_csv(fn)
                    print()
                    validInput = False

                elif confirm.lower() == "n":
                    clear_output()
                    validInput = True

                elif confirm.lower() == "exit":
                    clear_output()
                    print("Returning current dataframe state")
                    return df
                
                else:
                    print("Not a valid response")
                    print()
                
                
            count += 1
            
            if count % 100 == 0:
                print("Holy cow, that's "+str(count)+" narratives! Drink some water and look out the window for a moment.")
                print()
            
            elif count % 10 == 0:
                print("Woo! You've annotated "+str(count)+" narratives!")
                print()
                
            
                
                
    print("Finished looking for", phrase)
    return df

In [4]:
df = pd.read_csv(fn)
if 'Unnamed: 0' in list(df):
    df = df.drop(columns=['Unnamed: 0'])
    
print("Loaded the dataframe")
print()
print("Number of narratives containing \""+phrase+"\":", df[df['narrative_text'].str.lower().str.contains(phrase)].shape[0])
print("Number of unannotated narratives containing \""+phrase+"\":", df[(df['narrative_text'].str.lower().str.contains(phrase)) & df['confirm_healthy'].isnull()].shape[0])

/Users/schabdachj/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Loaded the dataframe

Number of narratives containing "glioma": 8232
Number of unannotated narratives containing "glioma": 6366


# Interactive Part

In [ ]:
print("Number of unannotated narratives:", df[df['confirm_healthy'].isnull()].shape[0])
df = markPhraseUnusable(df, fn, phrase)
print("Number of unannotated narratives:", df[df['confirm_healthy'].isnull()].shape[0])
df.to_csv(fn)